In [91]:
import torch
import pandas as pd
from torch_geometric.nn import HeteroConv, Linear, SAGEConv, to_hetero
import torch.nn.functional as F
import torch_geometric.transforms as T



In [92]:
class IdentityEncoder:
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

In [93]:
def load_node_csv(path, index_col, encoders=None, **kwargs):
    df = pd.read_csv(path, index_col=index_col, **kwargs)
    mapping = {index: i for i, index in enumerate(df.index.unique())}

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

In [94]:
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  attr_encoders=None,label_encoders = None ,**kwargs):
    df = pd.read_csv(path, **kwargs)

    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])

    edge_attr = None
    if attr_encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in attr_encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)
    if label_encoders is not None:
        edge_labels = [encoder(df[col]) for col, encoder in label_encoders.items()]
        edge_label = torch.cat(edge_labels, dim=-1)

    return edge_index, edge_attr, edge_label

In [95]:
customer_encoders = {col_name : IdentityEncoder(dtype=torch.float32) for col_name in ["n_orders","n_not_returned","n_partial_returned","n_fully_returned","n_products"]}
order_encoders = {col_name : IdentityEncoder(dtype=torch.float32) for col_name in ["label","n_products"]}
product_encoders = {col_name : IdentityEncoder(dtype=torch.float32) for col_name in ["n_orders", "n_returned"]}
customer_x, customer_mapping = load_node_csv('data/customer_node_attributes.csv', index_col='customer', encoders=customer_encoders)
order_x, order_mapping = load_node_csv('data/order_node_attributes.csv', index_col='order', encoders=order_encoders)
product_x, product_mapping = load_node_csv('data/product_node_attributes.csv', index_col='product', encoders=product_encoders)

In [96]:
customer_order_edge_attr_encoders = {col_name : IdentityEncoder(dtype=torch.float32) for col_name in ["n_products","train_masks","valid_masks","test_masks"]}
customer_order_edge_label_encoders = {"label" : IdentityEncoder(torch.long)}
customer_order_edge_index, customer_order_edge_attr, customer_order_edge_label = load_edge_csv('data/customer_edgelist.csv', 'customer', customer_mapping, 'order', order_mapping, attr_encoders=customer_order_edge_attr_encoders, label_encoders=customer_order_edge_label_encoders)
order_product_edge_index, order_product_edge_attr, order_product_edge_label = load_edge_csv('data/order_edgelist.csv', 'order', order_mapping, 'product', product_mapping, label_encoders={"label": IdentityEncoder(torch.long)})

In [97]:
from torch_geometric.data import HeteroData

data = HeteroData()

customer_mapping = []
order_mapping = []
item_mapping = []


data['order'].x = order_x
data['product'].x = product_x
data['customer'].x = customer_x
data['customer', 'order'].edge_index = customer_order_edge_index
data['customer', 'order'].edge_attr = customer_order_edge_attr
data['customer', 'order'].edge_label = customer_order_edge_label

data['order',  'product'].edge_index = order_product_edge_index
data['order',  'product'].edge_attr = order_product_edge_attr
data['order',  'product'].edge_label = order_product_edge_label

data = T.ToUndirected()(data)

print(data)

HeteroData(
  order={ x=[849185, 2] },
  product={ x=[58415, 2] },
  customer={ x=[342038, 5] },
  (customer, to, order)={
    edge_index=[2, 849185],
    edge_attr=[849185, 4],
    edge_label=[849185, 1],
  },
  (order, to, product)={
    edge_index=[2, 2666262],
    edge_label=[2666262, 1],
  },
  (order, rev_to, customer)={
    edge_index=[2, 849185],
    edge_attr=[849185, 4],
    edge_label=[849185, 1],
  },
  (product, rev_to, order)={
    edge_index=[2, 2666262],
    edge_label=[2666262, 1],
  }
)


In [98]:
# DOES NOT WORK FROM HERE

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x
    

model = GNN(hidden_channels=64, out_channels=3)
model = to_hetero(model, data.metadata(), aggr='sum')

In [99]:
model

GraphModule(
  (conv1): ModuleDict(
    (customer__to__order): SAGEConv((-1, -1), 64, aggr=mean)
    (order__to__product): SAGEConv((-1, -1), 64, aggr=mean)
    (order__rev_to__customer): SAGEConv((-1, -1), 64, aggr=mean)
    (product__rev_to__order): SAGEConv((-1, -1), 64, aggr=mean)
  )
  (conv2): ModuleDict(
    (customer__to__order): SAGEConv((-1, -1), 3, aggr=mean)
    (order__to__product): SAGEConv((-1, -1), 3, aggr=mean)
    (order__rev_to__customer): SAGEConv((-1, -1), 3, aggr=mean)
    (product__rev_to__order): SAGEConv((-1, -1), 3, aggr=mean)
  )
)

In [108]:
data['customer', 'order'].edge_label.T

tensor([[-1,  1,  2,  ...,  1, -1,  1]])

In [127]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    optimizer.zero_grad()
    output = model(data.x_dict, data.edge_index_dict,
                 )
    # edge_attr[:,-3] - train mask
    mask = data['customer', 'order'].edge_attr[:,-3].bool()
    pred = F.softmax(output["order"][mask], dim=-1)
    target = data['customer', 'order'].edge_label[mask].squeeze()
    print(output["order"][mask][0])
    loss = F.cross_entropy(pred, target)
    loss.backward()
    optimizer.step()
    return float(loss)

In [129]:
for epoch in range(1, 15):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')


tensor([ 26.1372, 308.6824,  75.3516], grad_fn=<SelectBackward0>)
Epoch: 01, Loss: 1.1567
tensor([ -7.7246, 344.7181, 135.9347], grad_fn=<SelectBackward0>)
Epoch: 02, Loss: 1.1567
tensor([-37.0440, 375.4637, 190.2463], grad_fn=<SelectBackward0>)
Epoch: 03, Loss: 1.1566
tensor([-62.9246, 400.9625, 239.0159], grad_fn=<SelectBackward0>)
Epoch: 04, Loss: 1.1563
tensor([-86.3450, 412.6218, 282.6101], grad_fn=<SelectBackward0>)
Epoch: 05, Loss: 1.2017
tensor([-104.6045,  458.6339,  248.4642], grad_fn=<SelectBackward0>)
Epoch: 06, Loss: 1.1566
tensor([-121.0403,  499.9644,  218.0501], grad_fn=<SelectBackward0>)
Epoch: 07, Loss: 1.1567
tensor([-135.8651,  537.5198,  191.0305], grad_fn=<SelectBackward0>)
Epoch: 08, Loss: 1.1567
tensor([-149.3281,  571.5295,  166.6568], grad_fn=<SelectBackward0>)
Epoch: 09, Loss: 1.1567
tensor([-161.5548,  602.3651,  144.6206], grad_fn=<SelectBackward0>)
Epoch: 10, Loss: 1.1567
tensor([-172.7263,  630.3967,  124.6418], grad_fn=<SelectBackward0>)
Epoch: 11, Loss:

In [114]:
# Example of target with class indices
input = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)
loss = F.cross_entropy(input, target)

print(input.shape, target.shape, loss)
# Example of target with class probabilities
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.randn(3, 5).softmax(dim=1)
# loss = F.cross_entropy(input, target)


torch.Size([3, 5]) torch.Size([3]) tensor(1.8807, grad_fn=<NllLossBackward0>)


In [49]:
data['customer', 'order'].edge_label.unique()


tensor([0., 1.])